In [6]:
import pandas as pd
import numpy as np
from tools import model_builder
from itertools import product
from dataset_management import get_dataset


pd.set_option("chained_assignment", None)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", 5000)
pd.set_option("display.width", 10000)

In [2]:
# Sanity checks on PriorBand-data
dataset=get_dataset(dataset_name="fig5/fig5_full")
dataset=dataset.loc[(dataset["used_fidelity"].isin(range(1,25)))&(dataset["prior"].isin(["at25","bad"]))]

builder=model_builder(dataset,"normalized_regret","algorithm","benchmark")
builder.test_seed_dependency()
builder.test_benchmark_information()
builder.test_fidelity(fidelity_var="used_fidelity")

✅ Loading data done                                                                                 
Simple model (49545.38) << Model with Seed-effect (49756.71)
Chi-Square: 422.66221730403777, P-Value: 0.0
Seed is a significant effect, likely influenced algorithms: []

Benchmark: LC-167190
Simple model (5937.06) << Model with Algorithm-effect (6137.96)
Chi-Square: 401.7876838870907, P-Value: 0.0
=> Benchmark LC-167190    is informative.

Benchmark: LC-126026
Simple model (8679.76) << Model with Algorithm-effect (8717.63)
Chi-Square: 75.7407856357313, P-Value: 0.0
=> Benchmark LC-126026    is informative.

Benchmark: LC-168330
Simple model (5895.78) << Model with Algorithm-effect (5990.89)
Chi-Square: 190.22010228375802, P-Value: 0.0
=> Benchmark LC-168330    is informative.

Benchmark: LC-168910
Simple model (4327.47) << Model with Algorithm-effect (4508.31)
Chi-Square: 361.6629492350985, P-Value: 0.0
=> Benchmark LC-168910    is informative.

Benchmark: PD1-LM1B
Simple model (5960.76

In [8]:
# Synthetic data

algos=["A-1","A-2","A-3"]
seeds=[str(x) for x in range(50)]
benchmarks=["B-0","B-1","B-2"]
budgets=range(1,11)
combination = list(product(algos, benchmarks, [0],seeds,budgets))


In [9]:
# In this dataset, one of the algorithms has a seed dependency
seed_df = pd.DataFrame(combination, columns=["algorithm", "benchmark", "value", "seed","budget"])
seed_df["value"] = seed_df.apply(lambda row:np.random.normal(int(row["seed"])*0.1 if row["algorithm"][2] in ["1"] and (int(row["seed"])%5==0 and row["seed"]!="0") else 2.5, 0.1 if row["algorithm"][2] in ["1"] else 0.55), axis=1)

builder=model_builder(seed_df)
builder.test_seed_dependency()

Simple model (-3725.63) << Model with Seed-effect (-2956.25)
Chi-Square: 1538.7509508900293, P-Value: 0.0
Seed is a significant effect, likely influenced algorithms: ['A-1']


['A-1']

In [10]:
# In this dataset, some benchmarks have algorithmic variation and some don't
benchmark_df = pd.DataFrame(combination, columns=["algorithm", "benchmark", "value", "seed","budget"])
benchmark_df["value"] = benchmark_df.apply(lambda row: np.random.normal(0.1*int(row["algorithm"][2])*int(row["benchmark"][2]), 0.01), axis=1)

builder=model_builder(benchmark_df)
builder.test_benchmark_information(rank_benchmarks=True)


Benchmark: B-0
Simple model (4782.04) == Model with Algorithm-effect (4783.5)
Chi-Square: 2.9292915607220493, P-Value: 0.2311598583386154
=> Benchmark B-0 is uninformative.

Benchmark: B-1
Simple model (1613.37) << Model with Algorithm-effect (4828.87)
Chi-Square: 6431.004016531817, P-Value: 0.0
=> Benchmark B-1 is informative.

Benchmark: B-2
Simple model (587.98) << Model with Algorithm-effect (4764.94)
Chi-Square: 8353.925176262983, P-Value: 0.0
=> Benchmark B-2 is informative.

  Name           Var       Std
0  B-0  1.042499e-08  0.000102
1  B-1  3.595527e-02  0.189619
2  B-2  1.433224e-01  0.378579
Benchmarks without algorithm variation: ['B-0']


({'B-0': False, 'B-1': True, 'B-2': True},
           Name           Var       Std
 algorithm  B-0  1.042499e-08  0.000102
 algorithm  B-1  3.595527e-02  0.189619
 algorithm  B-2  1.433224e-01  0.378579)

In [11]:
# In this dataset, the budget has a direct impact on the value
budget_df=pd.DataFrame(combination, columns=["algorithm", "benchmark", "value", "seed","budget"])
budget_df["value"] = budget_df.apply(lambda row:np.random.normal(0.1*np.random.normal(int(row["budget"])-5,6,1)[0], 0.2), axis=1)

builder=model_builder(budget_df)
builder.test_fidelity(fidelity_var="budget")

Simple model (-4682.55) << Model with Fidelity-effect (-4289.31)
Chi-Square: 786.4719513087948, P-Value: 0.0

Simple model (-4682.55) << Model with Fidelity-interaction-effect (-4288.38)
Chi-Square: 788.335417915785, P-Value: 0.0

Model with Fidelity-effect (-4289.31) == Model with Fidelity-interaction-effect (-4288.38)
Chi-Square: 1.8634666069901868, P-Value: 0.3938704213910704

=> Fidelity budget is both as simple and interaction effect significant, but as simple effect performs better.
